In [27]:
#pip install --upgrade pandas==1.3.5

In [1]:
import numpy as np
import pandas as pd
import os
import geopandas
from geocube.api.core import make_geocube
from pathlib import Path
from GeoDS.prospectivity import hyperparameterstuning
from GeoDS import hypercube
from GeoDS import utilities
from GeoDS.supervised import mapclass
from GeoDS.prospectivity import reporting 
from GeoDS.prospectivity import featureimportance
from GeoDS import eda
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from joblib import dump, load

import optuna
from optuna import pruners
from imblearn.pipeline import Pipeline

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, f1_score, precision_score
from sklearn.ensemble import RandomForestClassifier

from matplotlib import pyplot as plt
plt.rcParams["figure.facecolor"] = 'white'
plt.rcParams["axes.facecolor"] = 'white'
plt.rcParams["savefig.facecolor"] = 'white'

In [2]:
crs = 'epsg:26918'
AOI = 'Inputs/AOI/shape/AOI_geol.shp'
xRes = 5
yRes = 5
pixel_size = 5

# Random seed
random_state = 42

In [3]:
trial_name = 'Bufferize'

reporting_folder = os.path.join(trial_name, 'reporting/')
output_folder = os.path.join(trial_name, 'outputs/')
predictions_folder = os.path.join(trial_name, 'predictions/')
CatBoost_predictions_folder = os.path.join(predictions_folder, 'CatBoost_predictions/')
RF_predictions_folder = os.path.join(predictions_folder, 'RF_predictions/')
LGBM_predictions_folder = os.path.join(predictions_folder, 'LGBM_predictions/')
SVM_predictions_folder = os.path.join(predictions_folder, 'SVM_predictions/')

if not os.path.exists(reporting_folder):
    os.makedirs(reporting_folder)
        
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder) 

if not os.path.exists(CatBoost_predictions_folder):
    os.makedirs(CatBoost_predictions_folder)

if not os.path.exists(RF_predictions_folder):
    os.makedirs(RF_predictions_folder)
    
if not os.path.exists(LGBM_predictions_folder):
    os.makedirs(LGBM_predictions_folder)

if not os.path.exists(SVM_predictions_folder):
    os.makedirs(SVM_predictions_folder) 

In [4]:
def Path_Info(input_path):
    """
    Gives the name of a file, its extension and the path to host directory, for a given path.
    Parameters
    ----------
    input_path : str
        Path to a file
    Returns
    -------
    str
        name of the file (without extension)
    str
        extension of the file
    str
        host directory
    """
    path_obj = Path(input_path)
    # gives just the input name
    name = path_obj.stem
    extension = path_obj.suffix
    directory = str(path_obj.parent)

    return name, extension, directory


In [5]:
def dataframe_to_shapefile(df, output_shapefile, x_col, y_col, crs, columns=None):
    if(columns != None):
        if(type(columns) == list):
            columns.append(x_col)
            columns.append(y_col)
            df=df[columns]
        else:
            raise TypeError("The type of input argument \"columns\" should be a list.")
    x_empty_sum = df[x_col].isnull().sum()
    y_empty_sum = df[y_col].isnull().sum()
    if(x_empty_sum > 0 or y_empty_sum >0 ):
        print("Your input csv has %s x and %s y empty cells for the coordinates. These records will be dropped for the shapefile." % x_empty_sum, y_empty_sum)
        df.dropna(axis=0, how='any', subset=[x_col, y_col], inplace=True)
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df[x_col], df[y_col]), crs=crs)
    file, ext, directory = Path_Info(output_shapefile)
    if (os.path.exists(directory) != True):
        os.makedirs(directory)
    gdf.to_file(output_shapefile)
    return gdf

In [6]:
def csv_to_shapefile(input_csv, output_shapefile, x_col, y_col, crs, columns=None):
    df = pd.read_csv(input_csv)
    if(columns != None):
        if(type(columns) == list):
            columns.append(x_col)
            columns.append(y_col)
            df=df[columns]
        else:
            raise TypeError("The type of input argument \"columns\" should be a list.")
    x_empty_sum = df[x_col].isnull().sum()
    y_empty_sum = df[y_col].isnull().sum()
    if(x_empty_sum > 0 or y_empty_sum >0 ):
        print("Your input csv has %s x and %s y empty cells for the coordinates. These records will be dropped for the shapefile." % x_empty_sum, y_empty_sum)
        df.dropna(axis=0, how='any', subset=[x_col, y_col], inplace=True)
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df[x_col], df[y_col]), crs=crs)
    file, ext, directory = Path_Info(output_shapefile)
    if (os.path.exists(directory) != True):
        os.makedirs(directory)
    gdf.to_file(output_shapefile)
    return gdf

In [7]:
def binary_buffers_from_shapefile(input_shapefile, output_folder, crs, xRes, yRes, field_name='value', buffer_lst=[25, 50, 100], buffer_shape=1, verbose=False,
                             strict=False, keep_label=False):
    """
    Create a circle buffer around X-Y point data for shapefiles which values are 0 or 1. This function does the same as "BufferizeShapefile" but can handle large numbers
    of data such as DDH assays in active mines. It can also create a twin raster which instead of carrying
    the binary value, carries the originally attached ID meant for train/validation/test splitting.
    Parameters
    ----------
    input_shapefile : str
        input shapefile path
    output_folder : str
        output shapefile path
    field_name : str
        name of the column with the binary classification
    buffer_lst : list
        list of int, the buffer sizes
    buffer_shape : int, default=1
        value can be 1, 2, 3 for circle, flat, or square. see https://shapely.readthedocs.io/en/latest/manual.html#object.buffer
    strict : bool
        Wheter or not to enforce strict distance between points (i.e. further than buffer radius)
    keep_label = bool
        Wheter or not to generate the twin raster with original label data
    Returns
    -------
    None
    """
    # Test
    #test, pts = buffer_overlap_check(input_shapefile, max(buffer_lst), verbose)
    #if strict and not test:
    #    print('\n'.join([str(pt) for pt in pts]))
    #    raise Warning('Points too close')
    if (os.path.exists(output_folder) != True):
        os.makedirs(output_folder)
    in_filename, extension, directory = utilities.Path_Info(input_shapefile)
    gdf = geopandas.read_file(input_shapefile)
    if field_name not in gdf.columns:
        gdf[field_name] = 1
    gdf_1 = gdf[gdf[field_name] == 1]
    gdf_0 = gdf[gdf[field_name] == 0]
    for buffer in buffer_lst:
        # <BEGIN> New code to generate a tiff that has target# info
        # Just a copy of the original code adapted
        #if keep_label:
        #    # loop over all targets and buffer them
        #    # ideally vectorized but temporary
        #    pts_target = gdf[["geometry"]]
        #    result_dict = {}
        #    fields_list = []
        #    for target_num in gdf.index.values:
        #        gdf_target = gdf[gdf.index == target_num]
        #        buff_gdf = gdf_target.copy().drop(["geometry"], axis=1)
        #        buff_gdf["buffer"] = pts_target.buffer(buffer, cap_style=buffer_shape)
        #        buff_gdf = buff_gdf.rename(columns={"buffer": "geometry"})
        #
        #        union_target = buff_gdf.geometry.unary_union
        #        s_target = geopandas.GeoSeries(union_target, crs=crs)
        #
        #       field_target = 'target_' + str(target_num)
        #        fields_list.append(field_target)
        #        d_target = {
        #            field_target: target_num,
        #            "geometry": s_target
        #        }
        #        result_target = geopandas.GeoDataFrame(d_target)
        #        result_target = result_target.explode(index_parts=True)
        #        result_dict[target_num] = result_target
        #    # Unite all individual target buffered dataframes and keep
        #    # highest on overlap
        #    # if option strict=True, overlap is not permitted
        #    i = 0
        #    for key in result_dict:
        #        if i == 0:
        #            union_gdf_t = result_dict[key]
        #       else:
        #            result_gdf = result_dict[key]
        #             union_gdf_t = geopandas.overlay(
        #                union_gdf_t,
        #                result_gdf,
        #                how='union'
        #            )
        #        i = i + 1
        #    union_gdf_t['value'] = union_gdf_t.loc[:, fields_list].max(axis=1)
        #    out_grid_t = make_geocube(
        #        vector_data=union_gdf_t,
        #        output_crs=crs,
        #        measurements=['value'],
        #        fill=-9999,
        #        resolution=(xRes, yRes)
        #    )
        #    out_grid_t['value'].rio.to_raster(
        #        f"{output_folder}{in_filename}_{field_name}_b{str(buffer)}_target-label.tif"
        #    )
        # <\END>
        # Back to original code
        pts_1 = gdf_1[["geometry"]]
        buff_gdf_1 = gdf_1.copy().drop(["geometry"], axis=1)
        buff_gdf_1["buffer"] = pts_1.buffer(buffer, cap_style=buffer_shape)
        buff_gdf_1 = buff_gdf_1.rename(columns={"buffer": "geometry"})
        pts_0 = gdf_0[["geometry"]]
        buff_gdf_0 = gdf_0.copy().drop(["geometry"], axis=1)
        buff_gdf_0["buffer"] = pts_0.buffer(buffer, cap_style=buffer_shape)
        buff_gdf_0 = buff_gdf_0.rename(columns={"buffer": "geometry"})
        union_0 = buff_gdf_0.geometry.unary_union
        union_1 = buff_gdf_1.geometry.unary_union
        s_0 = geopandas.GeoSeries(union_0, crs=crs)
        s_1 = geopandas.GeoSeries(union_1, crs=crs)
        #s_0 = geopandas.GeoSeries(buff_gdf_0.geometry, crs=crs)
        #s_1 = geopandas.GeoSeries(buff_gdf_1.geometry, crs=crs)
        field_0 = field_name + "_0"
        d_0 = {
            field_0: 0,
            "geometry": s_0
        }
        result_0 = geopandas.GeoDataFrame(d_0)
        result_0 = result_0.explode(index_parts=True)
        result_0['target_id_0'] = result_0.apply(lambda row : row.name[1]+1,axis=1 )
        nb_rows_0 = len(result_0)
        #print(result_0)
        field_1 = field_name + "_1"
        d_1 = {
            field_1: 1,
            "geometry": s_1
        }
        result_1 = geopandas.GeoDataFrame(d_1)
        result_1 = result_1.explode(index_parts=True)
        result_1['target_id_1'] = result_1.apply(lambda row: row.name[1] + nb_rows_0 + 1, axis=1)
        print(result_1)
        union_gdf = geopandas.overlay(result_0, result_1, how='union', keep_geom_type=False)
        #union_gdf = union_gdf.explode(index_parts=True)
        union_gdf['value'] = union_gdf[[field_1, field_0]].max(axis=1)
        union_gdf['target_id'] = union_gdf.apply(lambda row: row['target_id_1'] if row['value'] == 1 else row['target_id_0'], axis=1)
        #print(union_gdf)
        #return union_gdf
        #union_gdf['target_id'] = union_gdf[['target_id_0', 'target_id_1']].max(axis=1)
        out_grid = make_geocube(vector_data=union_gdf,
                                output_crs=crs,
                                measurements=['value'],
                                fill=-9999,
                               resolution=(xRes, yRes))
        out_grid2 = make_geocube(vector_data=union_gdf,
                                output_crs=crs,
                                measurements=['target_id'],
                                fill=-9999,
                                resolution=(xRes, yRes))
        # Export the raster
        #out_grid['value'].rio.to_raster(f"{output_folder}{in_filename}_{field_name}_b{str(buffer)}.tif")
        outname = os.path.join(output_folder, in_filename + "_" +field_name+"_b"+str(buffer)+".tif")
        reproj_name = outname.replace('.tif', '_r.tif')
        out_grid['value'].rio.to_raster(outname)
        utilities.warp(outname, reproj_name, crs, xRes, yRes)
        os.remove(outname)
        os.rename(reproj_name, outname)
        print("Bufferization done for " + str(buffer) + " m")
        if(keep_label == True):
            outname2 = os.path.join(output_folder,
                                    in_filename + "_" + field_name + "_b" + str(buffer) + "_target_id.tif")
            reproj_name2 = outname2.replace('.tif', '_r.tif')
            out_grid2['target_id'].rio.to_raster(outname2)
            utilities.warp(outname2, reproj_name2, crs, xRes, yRes)
            os.remove(outname2)
            os.rename(reproj_name2, outname2)
            print("Target labels also saved.")
    return None

# dataframe to csv

In [14]:
input_geotiff = 'Inputs/Targets/Proper_Raster/CRE_all_Li_ne_and_pos.tif'
output_csv = 'Inputs/Targets/Shapes_v2/raster/CRE_all_Li_ne_and_pos.csv'

utilities.geotiff_to_csv(input_geotiff, output_csv)


Conversion from Geotiff to CSV successful. See Inputs/Targets/Shapes_v2/raster/CRE_all_Li_ne_and_pos.csv


In [27]:
df = pd.read_csv(output_csv)
df.head()

,X,Y,Z
0,386832.5,5701002.5,1.0
1,408877.5,5707687.5,0.0
2,408867.5,5707697.5,0.0
3,408902.5,5707717.5,0.0
4,408907.5,5707717.5,0.0


In [26]:
np.unique(df['Z'])

array([0., 1.])

# csv to shapefile

In [28]:
input_csv_list = ['Inputs/Targets/Shapes_v2/raster/CRE_all_Li_ne_and_pos.csv'
                 ]

output_shapefile_list = ['Inputs/Targets/Shapes_v2/raster/shp_to_buffer_v2/CRE_all_Li_ne_and_pos.shp'
                        ]

for i in range(len(input_csv_list)):
    csv_to_shapefile(input_csv = input_csv_list[i],
                 output_shapefile = output_shapefile_list[i],
                 x_col = 'X',
                 y_col = 'Y',
                 crs = crs,
                 columns=['Z']
                )

In [8]:
import glob
import os
from osgeo import gdal, gdalconst, ogr
from osgeo_utils import gdal2xyz

def build_stack(
    numerical_folders,
    categorical_folders,
    output_folder,
    aoi_shapefile,
    pixel_size = 30,
    nodata = -999999
):
    num_files_list = []
    cat_files_list = []
    for num in numerical_folders:
        num_files_list += glob.glob(os.path.join(num, '*.tif'))
    for cat in categorical_folders:
        cat_files_list += glob.glob(os.path.join(cat, '*.tif'))
    os.makedirs(output_folder, exist_ok=True)
    # Read AOI
    aoi_name = aoi_shapefile.split('/')[-1].split('.')[0]
    src_ds = ogr.Open(aoi_shapefile, 0)
    source_layer = src_ds.GetLayer()
    source_proj = source_layer.GetSpatialRef()
    xmin, xmax, ymin, ymax = source_layer.GetExtent()
    # Write WarpOptions
    sql = f'SELECT * FROM {aoi_name}'
    num_options = gdal.WarpOptions(
        format= 'GTiff',
        outputBounds= (xmin, ymin, xmax, ymax),
        xRes= pixel_size,
        yRes= pixel_size,
        targetAlignedPixels= True,
        dstSRS= source_proj,
        resampleAlg= 'cubic',
        outputType= gdal.GDT_Float32,
        dstNodata= nodata,
        multithread= True,
        cutlineDSName= aoi_shapefile,
        cutlineSQL= sql,
        cropToCutline= True
    )
    cat_options = gdal.WarpOptions(
        format= 'GTiff',
        outputBounds= (xmin, ymin, xmax, ymax),
        xRes= pixel_size,
        yRes= pixel_size,
        targetAlignedPixels= True,
        dstSRS= source_proj,
        resampleAlg= 'near',
        outputType= gdal.GDT_Int16,
        dstNodata= nodata,
        multithread= True,
        cutlineDSName= aoi_shapefile,
        cutlineSQL= sql,
        cropToCutline= True
    )
    stack_list = []
    header_list = ['x', 'y']
    # Warp every file in input folders
    for cat_file in cat_files_list:
        print(cat_file)
        filename = os.path.basename(cat_file).split('.')[0]
        in_path = os.path.join(cat_file)
        out_path = os.path.join(output_folder, f'{filename}.tif')
        gdal.Warp(out_path, in_path, options=cat_options)
        stack_list.append(out_path)
        header_list.append(filename)

    for num_file in num_files_list:
        print(num_file)
        filename = os.path.basename(num_file).split('.')[0]
        in_path = os.path.join(num_file)
        out_path = os.path.join(output_folder, f'{filename}.tif')
        gdal.Warp(out_path, in_path, options=num_options)
        stack_list.append(out_path)
        header_list.append(filename)
    # Build multiband VRT
    vrt = os.path.join(output_folder, 'stack.vrt')
    gdal.BuildVRT(vrt, stack_list, separate=True)


def read_vrt_to_df(vrt):
    """Convert vrt to DataFrame"""

    import numpy as np
    import pandas as pd
    import rasterio
    import rioxarray as rio

    # Extract feature names
    with rasterio.open(vrt) as ds:
        features_list = ds.files[1:]
        feats = [feature.split('.')[0].split('/')[-1] for feature in features_list]
    # Read data and flatten arrays
    with rio.open_rasterio(vrt, masked=True) as ds:
        x_coords = ds.coords['x'].values
        y_coords = ds.coords['y'].values
        data_array = ds.data
        l, m, n = data_array.shape
        data_flat = np.empty((m*n, l))
        for i in range(l):
            flat = data_array[i].flatten()
            data_flat[:, i] = flat
    # Extract coordinates
    i = 0
    coords = np.empty((m*n, 2))
    for y in y_coords:
        for x in x_coords:
            coords[i] = [x, y]
            i += 1
    # Concatenate and write df
    final_array = np.concatenate((coords, data_flat), axis=1)
    headers = ['x', 'y'] + feats
    del data_flat
    df = pd.DataFrame(data=final_array, columns=headers)
    return df

In [9]:
numerical_folders = ['Inputs/Targets/Shapes_v2/raster/']

categorical_folders = []

build_stack(
    numerical_folders,
    categorical_folders,
    output_folder,
    aoi_shapefile =  AOI,
    pixel_size = pixel_size,
    nodata = -999999
)

Inputs/Targets/Shapes_v2/raster/CRE_all_Li_ne_and_pos.tif


In [10]:
df = read_vrt_to_df(os.path.join(output_folder, 'stack.vrt'))
df.head()

,x,y,CRE_all_Li_ne_and_pos
0,384727.5,5760787.5,NaN
1,384732.5,5760787.5,NaN
2,384737.5,5760787.5,NaN
3,384742.5,5760787.5,NaN
4,384747.5,5760787.5,NaN


In [11]:
df = df.rename(columns={'CRE_all_Li_ne_and_pos': 'target'})
df.head()

,x,y,target
0,384727.5,5760787.5,NaN
1,384732.5,5760787.5,NaN
2,384737.5,5760787.5,NaN
3,384742.5,5760787.5,NaN
4,384747.5,5760787.5,NaN


In [12]:
np.unique(df.target, return_counts=True)

(array([ 1.,  2., nan]), array([      128,       240, 358840130]))

In [13]:
df['target'] = df['target'].apply(lambda x: x-2 if x in [2.] else x)

In [14]:
df.dropna(inplace=True)
df.head()

,x,y,target
20695603,507302.5,5756907.5,0.0
21254970,507147.5,5756802.5,0.0
21308238,507107.5,5756792.5,0.0
21547877,506592.5,5756747.5,0.0
22613324,506227.5,5756547.5,0.0


In [15]:
df.dtypes

x         float64
y         float64
target    float64
dtype: object

In [16]:
df['target'] = df['target'].apply(np.int64)
#df['target'] = df['target'].apply(np.float32)
np.unique(df['target'], return_counts=True)

(array([0, 1]), array([240, 128]))

In [17]:
df.dtypes

x         float64
y         float64
target      int64
dtype: object

In [18]:
# df = df.reset_index()

# dataframe to shapefile

In [19]:
output_shapefile = 'Inputs/Targets/Shapes_v2/raster/shp_to_buffer_v2/CRE_all_Li_ne_and_pos.shp' 

df_shp = dataframe_to_shapefile(df,
                 output_shapefile,
                 x_col = 'x',
                 y_col = 'y',
                 crs = crs,
                 columns=['target']
                )

In [20]:
np.unique(df_shp['target'], return_counts=True)

(array([0, 1]), array([240, 128]))

In [21]:
df_shp.tail()

,target,x,y,geometry
282740567,0,408902.5,5707717.5,POINT (408902.500 5707717.500)
282740568,0,408907.5,5707717.5,POINT (408907.500 5707717.500)
282847112,0,408867.5,5707697.5,POINT (408867.500 5707697.500)
282900390,0,408877.5,5707687.5,POINT (408877.500 5707687.500)
318510987,1,386832.5,5701002.5,POINT (386832.500 5701002.500)


# Targets: Bufferize

In [22]:
from GeoDS.featureengineering import featureengineer as feat_eng

In [23]:
input_shapefiles = [
        'Inputs/Targets/Shapes_v2/raster/shp_to_buffer_v2/CRE_all_Li_ne_and_pos.shp',
                ]
output_buffer = 'Inputs/Targets/Shapes_v2/buffers/'

for input_shapefile in input_shapefiles:
    binary_buffers_from_shapefile(
        input_shapefile,
        output_buffer,
        crs,
        xRes,
        yRes,
        field_name='target',
        buffer_lst=[10, 25, 50, 100],
        buffer_shape=1,
        verbose=False,
        strict=False,
        keep_label=True
        )

      target_1                                           geometry  target_id_1
0 0          1  POLYGON ((386840.230 5700996.156, 386839.571 5...          220
  1          1  POLYGON ((409170.230 5707891.156, 409169.571 5...          221
  2          1  POLYGON ((421995.230 5708251.156, 421994.571 5...          222
  3          1  POLYGON ((410930.230 5708391.156, 410929.571 5...          223
  4          1  POLYGON ((422130.230 5708511.156, 422129.571 5...          224
...        ...                                                ...          ...
  91         1  POLYGON ((482120.230 5743981.156, 482119.571 5...          311
  92         1  POLYGON ((482275.230 5744111.156, 482274.571 5...          312
  93         1  POLYGON ((482740.230 5744431.156, 482739.571 5...          313
  94         1  POLYGON ((482659.597 5744582.931, 482658.673 5...          314
  95         1  POLYGON ((491315.230 5752481.156, 491314.571 5...          315

[96 rows x 3 columns]
Warp completed. See Inputs/Ta

Target labels also saved.


In [19]:
input_raster = 'Inputs/Targets/Shapes_v2/raster/CRE_all_Li_ne_and_pos.tif'
dst_shapefile = 'Inputs/Targets/Shapes_v2/raster/shp_to_buffer/CRE_all_Li_ne_and_pos.shp'

utilities.raster_to_shapefile(input_raster, dst_shapefile)
                    

CRE_all_Li_ne_and_pos.shp
Conversion Done. See Inputs/Targets/Shapes_v2/raster/shp_to_buffer/CRE_all_Li_ne_and_pos.shp


In [26]:
input_shapefile_directory = 'Inputs/Targets/Shapes_v2/'
input_shapefile_name = 'CRE_all_Li_ne_and_pos.shp'

input_shapefile = os.path.join(input_shapefile_directory,input_shapefile_name)
output_folder_buffer = 'Inputs/Targets/Shapes_v2/buffers'  

feat = feat_eng.FeatureEngineer(crs, AOI, pixel_size)

feat_eng.bufferize_lineaments(input_shapefile, 
                     output_folder_buffer, 
                     AOI, 
                     buffers=[10, 25, 50, 100])

AttributeError: 'NoneType' object has no attribute 'Buffer'

In [20]:
input_shapefile_directory = 'Inputs/Targets/Shapes_v2/raster/shp_to_buffer/'
input_shapefile_name = 'CRE_all_Li_ne_and_pos.shp'

input_shapefile = os.path.join(input_shapefile_directory,input_shapefile_name)
output_folder_buffer = 'Inputs/Targets/Shapes_v2/buffers'  

feat_eng.binary_buffers_from_shapefile(input_shapefile, 
                                   output_folder_buffer, 
                                   crs = crs, 
                                   xRes = pixel_size,
                                   yRes = pixel_size,
                                   field_name='Type',
                                   buffer_lst=[10, 25, 50, 100],
                                   buffer_shape=1,
                                   verbose=False,
                                   strict=False,
                                   keep_label=False)

AttributeError: 'Polygon' object has no attribute 'x'